# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
# for i in range(1, 60):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch
from maddpg import MADDPG
from buffer import ReplayBuffer
from collections import namedtuple, deque
from utilities import transpose_list, transpose_to_tensor

In [ ]:
layer_params = {
    'in_actor' : 24, # Observation vector size
    'hidden_in_actor' : 256,
    'hidden_out_actor' : 128,
    'out_actor' : 2, # Action vector size
    'in_critic' : 48, # 24 * 2 + 2 * 2 (Num of agents * Action size)
    'hidden_in_critic' : 256,
    'hidden_out_critic' : 128, 
    'lr_actor' : 1e-4,
    'lr_critic' : 3e-4
}

In [ ]:
maddpg = MADDPG(num_agents = 2, layer_params = layer_params,  discount_factor=0.99, tau=0.02)

In [ ]:
# #Constants
# BUFFER_SIZE = int(1e5)  # replay buffer size
# BATCH_SIZE = 256        # minibatch size
# GAMMA = 0.99            # discount factor
# TAU = 1e-3              # for soft update of target parameters
# LR_ACTOR = 1e-4 #3e-5 #1e-4         # learning rate of the actor 
# LR_CRITIC = 3e-4 #3e-5 #1e-4        # learning rate of the critic
# WEIGHT_DECAY_actor = 0.0 #3e-4 #0        # L2 weight decay
# WEIGHT_DECAY_critic = 0.0 #1e-6 #0        # L2 weight decay
# #to decay exploration as it learns
# NOISE_START=1.0
# NOISE_END=0.1
# NOISE_REDUCTION=0.999
EPISODES_BEFORE_TRAINING = 300
NUM_LEARN_STEPS_PER_ENV_STEP = 3
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# seeding(seed=2)
# state_size=env_info.vector_observations.shape[1]
# action_size=brain.vector_action_space_size
# num_agents=env_info.vector_observations.shape[0]
# MADDPG_obj = MADDPG(state_size=state_size, action_size=action_size, num_agents=num_agents)
###MADDPG_obj.load_maddpg() #load the local network weights

In [ ]:
#Training
def MADDPG_Training(n_episodes=2500, episode_t=1000):
    batch_size = 10
    scores_deque = deque(maxlen=100)
    scores_list = []
    scores_list_100_avg = []
    buffer = ReplayBuffer(int(5000*episode_t), batch_size)
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment
        states = env_info.vector_observations                   # get the current states (for all agents)
        maddpg.reset() #reset the MADDPG_obj OU Noise
        scores = np.zeros(num_agents)                          # initialize the score (for each agent in MADDPG)
        num_steps = 0
        for _ in range(episode_t):
            actions = maddpg.act(torch.from_numpy(states).float())
            actions = torch.stack(actions).cpu().detach().numpy()
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent in MADDPG)
            rewards = env_info.rewards                         # get rewards (for each agent in MADDPG)
            dones = env_info.local_done                        # see if episode finished
            scores += rewards                                  # update the score (for each agent in MADDPG)
            full_states = np.reshape(states, newshape=(-1))
            full_next_states = np.reshape(next_states, newshape=(-1))
            buffer.add(full_states, states, actions, rewards, full_next_states, next_states, dones)
#             maddpg.step(i_episode, states, actions, rewards, next_states, dones) #train the MADDPG_obj           
            

            # Learn, if enough samples are available in memory
            if len(buffer) > batch_size and i_episode > EPISODES_BEFORE_TRAINING:
                for _ in range(10):#NUM_LEARN_STEPS_PER_ENV_STEP): #learn multiple times at every step
                    for agent_no in range(num_agents):
                        samples = buffer.sample()
                        maddpg.update(samples, agent_no)
                    maddpg.update_targets() #soft update the target network towards the actual networks 
                
            states = next_states                               # roll over states to next time step
            num_steps += 1
            if np.any(dones):                                  # exit loop if episode finished
                break
        
#         if len(buffer) > batch_size:
#             for a_i in range(num_agents):
#                 samples = buffer.sample()
#                 maddpg.update(samples, a_i)
#             maddpg.update_targets() #soft update the target network towards the actual networks 
        scores_deque.append(np.max(scores))
        scores_list.append(np.max(scores))
        scores_list_100_avg.append(np.mean(scores_deque))
        
        print('Episode {}\tAverage Score: {:.2f}\tCurrent Score: {}'.format(i_episode, np.mean(scores_deque), np.max(scores)))
#         print('Noise Scaling: {}, Memory size: {} and Num Steps: {}'.format(MADDPG_obj.maddpg_agents[0].noise_scale, len(MADDPG_obj.memory), num_steps))
        
        if i_episode % 500 == 0:
            maddpg.save_maddpg()
            print('Saved Model: Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
        if np.mean(scores_deque) > 1.0 and len(scores_deque) >= 100:
            maddpg.save_maddpg()
            print('Saved Model: Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            break
            
    return scores_list, scores_list_100_avg

In [ ]:
scores, scores_avg = MADDPG_Training()